# Projet de Machine Learning

## Démarche

Dans ce projet, on va essayer de prédire les résultats du second tour des éléction de 2022.
Pour commencer, on a importer le dataset des résultats du second tour qui nous permet d'avoir des éléments de comparaison.
On essaye ensuite de trouver des datasets pour permettre de faire des prédictions.
On a trouvé un dataset sur le prix des loyer par maison et par appartement en fonction des communes pour savoir si on pouvait trouver une corélation. On a également trouvé un dataset avec les coordonés géographiques des communes comme proposé dans le sujet.
On a une précision de 50%. On peut imaginer qu'il y a une corélation, mais elle n'est pas suffisante.
On pense qu'il faudrait compléter les prix des loyers avec un revenu moyen par habitant en fonction des communes.
On pense aussi qu'il faudrait trouver l'age moyen par communes.
On a également fait le choix de ne télécharger aucun dataset pour permetre un dépot plus simple du projet. Cela indique également directement les sources utilisées. Ce choix est purement pratique. Cela implique cependant un temps d'éxécution plus lent.

In [46]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.regression.linear_model import OLS
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LassoCV
import numpy as np

## Preprocessing

In [21]:
data = pd.read_excel('https://www.data.gouv.fr/fr/datasets/r/06d9816c-1b87-498d-985e-f312acee4f51')

df = data.copy()
df = df.drop(df.columns[[26, 27, 29, 31, 32]], axis=1)
df = df.drop(['Code du département', 'N°Panneau', 'Sexe', 
              'Prénom', '% Abs/Ins', '% Vot/Ins', '% Blancs/Ins', '% Blancs/Vot', '% Nuls/Ins', 
              '% Nuls/Vot', '% Exp/Ins', '% Exp/Vot', '% Voix/Ins'], axis=1,)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

loyers_appart = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/8fac6fb7-cd07-4747-8e0b-b101c476f0da', encoding='iso-8859-1', sep=';')
loyers_appart.head(10)
loyers_maison = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/cf39b7e5-12fb-48e9-9ee7-b7e5496dcb75', encoding='iso-8859-1', sep=';')

# Convertie les colonnes 'loypredm2' en float
loyers_appart['loypredm2'] = pd.to_numeric(loyers_appart['loypredm2'].str.replace(',', '.'))
loyers_appart['loypredm2'] = pd.to_numeric(loyers_appart['loypredm2'])
loyers_maison['loypredm2'] = pd.to_numeric(loyers_maison['loypredm2'].str.replace(',', '.'))
loyers_maison['loypredm2'] = pd.to_numeric(loyers_maison['loypredm2'])

loyers_appart['upr.IPm2'] = pd.to_numeric(loyers_appart['upr.IPm2'].str.replace(',', '.'))
loyers_maison['upr.IPm2'] = pd.to_numeric(loyers_maison['upr.IPm2'].str.replace(',', '.'))
loyers_appart['upr.IPm2'] = pd.to_numeric(loyers_appart['upr.IPm2'])
loyers_maison['upr.IPm2'] = pd.to_numeric(loyers_maison['upr.IPm2'])

loyers_appart['lwr.IPm2'] = pd.to_numeric(loyers_appart['lwr.IPm2'].str.replace(',', '.'))
loyers_maison['lwr.IPm2'] = pd.to_numeric(loyers_maison['lwr.IPm2'].str.replace(',', '.'))
loyers_appart['lwr.IPm2'] = pd.to_numeric(loyers_appart['lwr.IPm2'])
loyers_maison['lwr.IPm2'] = pd.to_numeric(loyers_maison['lwr.IPm2'])

# Fusionner les deux DataFrames d'origine en un seul DataFrame
loyers = pd.merge(loyers_appart, loyers_maison, on='LIBGEO')

# Calculer la moyenne des loyers prédits pour les appartements et les maisons ligne par ligne
# loyers.head()
loyers['loypredm2_moyen'] = (loyers['loypredm2_x'] + loyers['loypredm2_y']) / 2
loyers['lwr.IPm2'] = (loyers['lwr.IPm2_x'] + loyers['lwr.IPm2_y']) / 2
loyers['upr.IPm2'] = (loyers['upr.IPm2_x'] + loyers['upr.IPm2_y']) / 2

# Créer un nouveau DataFrame avec les colonnes 'LIBGEO' et 'loypredm2_moyen'
loyers_moyen = loyers[['LIBGEO', 'loypredm2_moyen', 'lwr.IPm2', 'upr.IPm2']]

df_merged = pd.merge(df, loyers_moyen, left_on='Libellé de la commune', right_on='LIBGEO', how='left')
df_merged = df_merged.drop(['LIBGEO'], axis=1)

df_merged['Majorité_Macron'] = (df_merged['Voix'] > (df_merged['Exprimés'] - df_merged['Blancs'] + df_merged['Nuls']) / 2).astype(int)
# Pas forcément bien traité car si nombre de voix egales, on met que c'est pas la majorité de Le Pen

df_merged_clean = df_merged.dropna(subset=['loypredm2_moyen', 'lwr.IPm2', 'upr.IPm2'])

# Séparer les données en un ensemble d'entraînement et un ensemble de test
X = df_merged_clean[['loypredm2_moyen', 'lwr.IPm2', 'upr.IPm2']]
y = df_merged_clean['Majorité_Macron']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

df_communes = pd.read_csv('https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/georef-france-commune/exports/csv', sep = ';')

columns_to_keep = ['geo_point_2d', 'com_name', 'com_uu2020_status', 'com_in_ctu', 'com_is_mountain_area']
df_communes = df_communes.drop(columns=[col for col in df_communes.columns if col not in columns_to_keep], axis=1)

df_merged2 = pd.merge(df_merged_clean, df_communes, left_on='Libellé de la commune', right_on='com_name', how='left')
df_merged2 = df_merged2.drop(['com_name'], axis=1)

df_merged2[['latitude', 'longitude']] = df_merged2['geo_point_2d'].str.split(', ', expand=True).astype(float)

df_merged2['com_uu2020_status'] = df_merged2['com_uu2020_status'].apply(lambda x: 0 if x == 'hors unité urbaine' else 1)
df_merged2['com_in_ctu'] = df_merged2['com_in_ctu'].apply(lambda x: 0 if x == 'Non' else 1)
df_merged2['com_is_mountain_area'] = df_merged2['com_is_mountain_area'].apply(lambda x: 0 if x == 'Non' else 1)


df_merged2 = df_merged2.drop('geo_point_2d', axis=1)

df_merged2_cleaned = df_merged2.dropna(subset=['latitude', 'longitude', 'com_uu2020_status', 'com_in_ctu', 'com_is_mountain_area'])

df_test = pd.read_csv('data.csv', sep = ';')

df_merged3 = pd.merge(df_test, df_merged2_cleaned, left_on='Libellé', right_on='Libellé de la commune', how='left')

df_merged3 = df_merged3.drop(['Libellé'], axis=1)

emploi_H = "Taux d'emploi des hommes de 25-54 ans 2020"
emploi_F = "Taux d'emploi des femmes de 25-54 ans 2020"

etude_max_brevet = "Part des pers., dont le diplôme le plus élevé est le bepc ou le brevet, dans la pop. non scolarisée de 15 ans ou + 2020"
etude_max_bac2 = "Part des diplômés d'un BAC+2 dans la pop. non scolarisée de 15 ans ou + 2020"
etude_max_bac3_4 = "Part des diplômés d'un BAC+3  ou BAC+4 dans la pop. non scolarisée de 15 ans ou + 2020"
etude_max_bac5 = "Part des diplômés d'un BAC+5 ou plus dans la pop. non scolarisée de 15 ans ou + 2020"
part_loc_HLM = "Part des locataires HLM dans les rés. principales 2020"
age25_64 = "Part des pers. âgées de 25 à 64 ans 2020"
age65plus = "Part des pers. âgées de 65 ans ou + 2020"
creation_entreprise = "Créations d'entreprises (en nombre) 2022"


df_merged3 = df_merged3.dropna(subset=[emploi_H, emploi_F, etude_max_brevet, etude_max_bac2, etude_max_bac3_4, etude_max_bac5, part_loc_HLM, age25_64, age65plus, creation_entreprise])

df_cleaned = df_merged3.dropna()

Le dataset ``data.csv`` provient du site de l'insee: https://statistiques-locales.insee.fr/ ou l'on a combinée plusieurs types de données

## Suppression des valeurs NaN et des apostrophes dans les headers

In [22]:
# Supprimer les apostrophes des noms de colonnes dans le DataFrame nettoyé
df_cleaned.columns = [col.replace("'", "") for col in df_cleaned.columns]

# Liste des colonnes à convertir en float
columns_to_convert = [
    'Nb de pers. âgées de 65 ans ou + 2020',
    'Taux demploi des femmes de 25-54 ans 2020',
    'Taux demploi des hommes de 25-54 ans 2020',
    'Part des pers., dont le diplôme le plus élevé est le bepc ou le brevet, dans la pop. non scolarisée de 15 ans ou + 2020',
    'Part des diplômés dun BAC+2 dans la pop. non scolarisée de 15 ans ou + 2020',
    'Part des diplômés dun BAC+3  ou BAC+4 dans la pop. non scolarisée de 15 ans ou + 2020',
    'Part des diplômés dun BAC+5 ou plus dans la pop. non scolarisée de 15 ans ou + 2020',
    'Part des locataires HLM dans les rés. principales 2020',
    'Part des pers. âgées de 25 à 64 ans 2020',
    'Part des pers. âgées de 65 ans ou + 2020',
    'Créations dentreprises (en nombre) 2022'
]

# Liste des valeurs non numériques qui doivent être converties en NaN
non_numeric_values = ['', 'N/A - résultat non disponible', 'N/A - division par 0']

# Remplacer les valeurs non numériques par NaN
df_cleaned[columns_to_convert] = df_cleaned[columns_to_convert].replace(non_numeric_values, np.nan)

# Vous pouvez choisir de remplir les NaN par une valeur par défaut ou de les laisser tels quels selon vos besoins
# Par exemple, remplacer par 0 (vous pouvez choisir une autre valeur si nécessaire)
default_value = 0
df_cleaned[columns_to_convert] = df_cleaned[columns_to_convert].fillna(default_value)

# Convertir les colonnes en float
df_cleaned[columns_to_convert] = df_cleaned[columns_to_convert].astype(float)

# Affichage des informations pour vérification après conversion
print(df_cleaned[columns_to_convert].info())


/tmp/cache-anagarnier/xpython_418644/2523530882.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/cache-anagarnier/xpython_418644/2523530882.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/cache-anagarnier/xpython_418644/2523530882.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2099073 entries, 0 to 2100367
Data columns (total 11 columns):
 #   Column                                                                                                                   Dtype  
---  ------                                                                                                                   -----  
 0   Nb de pers. âgées de 65 ans ou + 2020                                                                                    float64
 1   Taux demploi des femmes de 25-54 ans 2020                                                                                float64
 2   Taux demploi des hommes de 25-54 ans 2020                                                                                float64
 3   Part des pers., dont le diplôme le plus élevé est le bepc ou le brevet, dans la pop. non scolarisée de 15 ans ou + 2020  float64
 4   Part des diplômés dun BAC+2 dans la pop. non scolarisée de 15 ans ou + 2020   

In [23]:
df_cleaned.head()

,Code,Nb de pers. âgées de 65 ans ou + 2020,Taux demploi des femmes de 25-54 ans 2020,Taux demploi des hommes de 25-54 ans 2020,"Part des pers., dont le diplôme le plus élevé est le bepc ou le brevet, dans la pop. non scolarisée de 15 ans ou + 2020",Part des diplômés dun BAC+2 dans la pop. non scolarisée de 15 ans ou + 2020,Part des diplômés dun BAC+3 ou BAC+4 dans la pop. non scolarisée de 15 ans ou + 2020,Part des diplômés dun BAC+5 ou plus dans la pop. non scolarisée de 15 ans ou + 2020,Part des locataires HLM dans les rés. principales 2020,Part des pers. âgées de 25 à 64 ans 2020,Part des pers. âgées de 65 ans ou + 2020,Créations dentreprises (en nombre) 2022,Libellé du département,Code de la commune,Libellé de la commune,Etat saisie,Inscrits,Abstentions,Votants,Blancs,Nuls,Exprimés,Nom,Voix,% Voix/Exp,Unnamed: 28,Unnamed: 30,loypredm2_moyen,lwr.IPm2,upr.IPm2,Majorité_Macron,com_uu2020_status,com_in_ctu,com_is_mountain_area,latitude,longitude
0,01001,158.0,88.4,95.1,5.2,13.7,9.7,6.4,3.9,53.4,19.6,17.0,Ain,1.0,L'Abergement-Clémenciat,Complet,643.0,146.0,497.0,42.0,5.0,450.0,MACRON,237.0,52.67,LE PEN,213.0,8.782018,6.997717,11.022057,1.0,0.0,0.0,0.0,46.153720,4.925852
1,01002,44.0,94.7,98.5,2.9,15.9,15.2,13.2,0.0,51.5,16.8,3.0,Ain,2.0,L'Abergement-de-Varey,Complet,213.0,45.0,168.0,23.0,3.0,142.0,MACRON,94.0,66.20,LE PEN,48.0,8.038375,6.196591,10.428710,1.0,0.0,0.0,1.0,46.009605,5.428088
2,01004,2585.0,71.1,87.7,6.3,12.8,7.6,7.0,22.2,50.3,18.1,218.0,Ain,4.0,Ambérieu-en-Bugey,Complet,8763.0,2448.0,6315.0,461.0,165.0,5689.0,MACRON,3080.0,54.14,LE PEN,2609.0,9.242846,7.078619,12.074234,1.0,1.0,0.0,0.0,45.961049,5.372277
3,01005,288.0,86.4,94.2,5.5,14.1,7.9,6.6,9.2,54.9,16.2,32.0,Ain,5.0,Ambérieux-en-Dombes,Complet,1282.0,266.0,1016.0,47.0,15.0,954.0,MACRON,451.0,47.27,LE PEN,503.0,9.205943,7.135369,11.882300,0.0,0.0,0.0,0.0,45.996163,4.911970
4,01006,32.0,88.2,78.3,7.9,11.9,10.9,5.9,1.8,56.1,28.1,2.0,Ain,6.0,Ambléon,Complet,103.0,19.0,84.0,12.0,0.0,72.0,MACRON,45.0,62.50,LE PEN,27.0,8.512610,6.769867,10.704187,1.0,0.0,0.0,1.0,45.749886,5.594583


## Régression Linéaire et Calcule de la précision

In [43]:
X3 = df_cleaned[['Nb de pers. âgées de 65 ans ou + 2020', 
                 'Taux demploi des femmes de 25-54 ans 2020', 
                 'Taux demploi des hommes de 25-54 ans 2020',
                 'Part des pers., dont le diplôme le plus élevé est le bepc ou le brevet, dans la pop. non scolarisée de 15 ans ou + 2020',
                 'Part des diplômés dun BAC+2 dans la pop. non scolarisée de 15 ans ou + 2020',
                 'Part des diplômés dun BAC+3  ou BAC+4 dans la pop. non scolarisée de 15 ans ou + 2020',
                 'Part des diplômés dun BAC+5 ou plus dans la pop. non scolarisée de 15 ans ou + 2020',
                 'Part des locataires HLM dans les rés. principales 2020',
                 'Part des pers. âgées de 25 à 64 ans 2020',
                 'Part des pers. âgées de 65 ans ou + 2020',
                 'Créations dentreprises (en nombre) 2022',
                 'loypredm2_moyen', 
                 'upr.IPm2', 
                 'latitude', 
                 'longitude', 
                 'com_uu2020_status', 
                 'com_is_mountain_area'
                 ]]
y3 = df_cleaned['Majorité_Macron']

X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3, test_size=0.2, random_state=0)

X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3, test_size=0.2, random_state=0)


model_ols = sm.OLS(y_train3, X_train3).fit()


print(model_ols.summary())

                                 OLS Regression Results                                
Dep. Variable:        Majorité_Macron   R-squared (uncentered):                   0.545
Model:                            OLS   Adj. R-squared (uncentered):              0.545
Method:                 Least Squares   F-statistic:                          1.181e+05
Date:                Mon, 22 Apr 2024   Prob (F-statistic):                        0.00
Time:                        18:28:36   Log-Likelihood:                     -1.2078e+06
No. Observations:             1679258   AIC:                                  2.416e+06
Df Residuals:                 1679241   BIC:                                  2.416e+06
Df Model:                          17                                                  
Covariance Type:            nonrobust                                                  
                                                                                                                        

In [44]:
means = X_train3.mean()
X_train3_centered = X_train3 - means
X_test3_centered = X_test3 - means

print("Number of samples in X_train:", X_train3_centered.shape[0])
print("Number of samples in y_train:", y_train3.shape[0])

Number of samples in X_train: 1679258
Number of samples in y_train: 1679258


In [47]:
lasso_cv = LassoCV(alphas=np.logspace(-6, 1, 100), cv=5, random_state=0)

lasso_cv.fit(X_train3_centered, y_train3)

# Ajuster le modèle Lasso sur les données centrées
print("Best alpha using built-in LassoCV: %f" % lasso_cv.alpha_)

# Evaluate the model on the test set
lasso_cv_score = lasso_cv.score(X_test3_centered, y_test3)
print('Précision du modèle LassoCV :', lasso_cv_score)

# Coefficients
print('Coefficients du modèle LassoCV:', lasso_cv.coef_)


Best alpha using built-in LassoCV: 0.000001
Précision du modèle LassoCV : 0.007149847752907501
Coefficients du modèle LassoCV: [ 7.15722329e-06  1.30751895e-03 -4.95935929e-04 -3.45968540e-03
  5.18238232e-03  2.79580294e-03  1.17922887e-03  1.53881906e-03
 -1.48723676e-03  1.85931804e-03 -5.76255784e-05 -1.41059339e-02
  7.35699205e-03  2.30780725e-03  8.37591126e-04  2.88287345e-02
  4.23485996e-02]


## Difficultés rencontrées

Le plus compliqué dans ce projet pour nous à été la recherche de datasets pertinents.
Il semble y avoir peu de datasets sur le thème qui nous intéresse d'explorer (age et situation financière).
On a pu rencontrer quelques datasets aussi vraiment incomplets ce qui ne permettait pas de faire des prédictions sur un nombre suffisant de communes.
La fusion de plusieurs datasets etait aussi un obstacle assez conséquent surtout quand on a fait des moyennes de plusieurs datasets notement sur les loyers où on a fait une moyenne des loyers maison et appartement compris. On justifie ça par le prix du logement en général par ville. On aurait pu aussi simplement ne pas faire de moyenne mais cela me semblait plus cohérent dans le sens où l'on souhaite avoir d'autres données pour appuyer notre prédiction.
On sait que cela est dit dans le sujet, mais on a du mal a avoir une bonne précision sur notre modèle. On pense donc que nos datasets employés ne sont pas forcément pertinent pour déterminer un penchant politique de chaque commune.